## Updating collection schema

We discussed the challenges with updating a profile schema in the last lesson.

The same applies for collections!

We want the ability to update the collection with new memories as well as update existing memories in the collection.

Now we'll show that [Trustcall](https://github.com/hinthornw/trustcall) can be also used to update a collection.

This enables both addition of new memories as well as [updating existing memories in the collection](https://github.com/hinthornw/trustcall?tab=readme-ov-file#simultanous-updates--insertions
).

Let's define a new extractor with Trustcall.

As before, we provide the schema for each memory, `Memory`.  

But, we can supply `enable_inserts=True` to allow the extractor to insert new memories to the collection.

In [1]:
from langchain_groq import ChatGroq
import os
from dotenv import load_dotenv

load_dotenv()
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")
model = ChatGroq(model="meta-llama/llama-4-scout-17b-16e-instruct", temperature=0)

In [2]:
from pydantic import BaseModel, Field
class Memory(BaseModel):
    """This is the memory field where store the collection of user data like his interested, his name, and so one. in one word all about the user info."""
    content: str = Field(description="The main content of the memory. User expressed interested in learning Programming.")

In [3]:
from trustcall import create_extractor

trustcall_extractor = create_extractor(
    model,
    tools=[Memory],
    tool_choice="Memory",
    enable_inserts=True,
    
)



In [4]:
from langchain_core.messages import HumanMessage, SystemMessage, AIMessage

# Instruction
instruction = """Extract memories from the following conversation:"""

# Conversation
conversation = [HumanMessage(content="Hi, I'm Al Amin"),
                AIMessage(content="Nice to meet you, Al Amin."),
                HumanMessage(content="This morning I had a nice bike ride in Dhaka.")]


result = trustcall_extractor.invoke(
    {
        "messages": [SystemMessage(content=instruction)]+conversation
    }
)

In [5]:
for m in result['messages']:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_g0zg)
 Call ID: call_g0zg
  Args:
    content: Al Amin had a bike ride in Dhaka.


In [6]:
for m in result['responses']:
    print(m.model_dump())

{'content': 'Al Amin had a bike ride in Dhaka.'}


In [7]:
for m in result['response_metadata']:
    print(m)

{'id': 'call_g0zg'}


### **Update the memory**

In [8]:
# Update the conversation
updated_conversation = [AIMessage(content="That's great, did you do after?"),
                        HumanMessage(content="I went to Tartine and ate a croissant."),
                        AIMessage(content="What else is on your mind?"),
                        HumanMessage(content="I was thinking about my Japan, and going back this winter!"),]

# Update the instruction
system_msg = """Update existing memories and create new ones based on the following conversation:"""

In [9]:
## Extracting the existing memory.
tool_name = "Memory"
existing_memories = [(str(i), tool_name, memory.model_dump()) for i, memory in enumerate(result['responses'])] if result['responses'] else None
existing_memories

[('0', 'Memory', {'content': 'Al Amin had a bike ride in Dhaka.'})]

In [10]:
## Update the memory
result = trustcall_extractor.invoke(
    {
        "messages": updated_conversation,
        "existing": existing_memories
    }
)

In [11]:
# Messages from the model indicate two tool calls were made
for m in result["messages"]:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  Memory (call_q4qp)
 Call ID: call_q4qp
  Args:
    content: Al Amin had a bike ride in Dhaka. I went to Tartine and ate a croissant. I was thinking about my Japan, and going back this winter!


In [12]:
for m in result["responses"]:
    print(m)

content='Al Amin had a bike ride in Dhaka. I went to Tartine and ate a croissant. I was thinking about my Japan, and going back this winter!'


In [13]:
# Metadata contains the tool call
for m in result["response_metadata"]:
    print(m)

{'id': 'call_q4qp'}
